In [ ]:
# 1. 환경 설정 및 라이브러리 로드
!pip install -q -U torch transformers peft bitsandbytes accelerate pandas datasets evaluate rouge_score

In [1]:
from evaluate import load
r = load("rouge")
print(r.compute(predictions=["이것은 테스트 요약입니다."], references=["이것은 테스트 요약입니다."]))

/opt/conda/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 'Could not load this library: /opt/conda/lib/python3.10/site-packages/torchvision/image.so'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


{'rouge1': 0.0, 'rouge2': 0.0, 'rougeL': 0.0, 'rougeLsum': 0.0}


In [2]:
import os
import torch
import pandas as pd
from tqdm import tqdm
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import PeftModel

# 설정
CONF = {
    "base_model": "upstage/SOLAR-10.7B-Instruct-v1.0",
    "adapter_path": "./results_solar",  # 학습된 어댑터 경로
    "data_path": "./data/",
    "seed": 42
}

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


## 2. 모델 및 토크나이저 로드 (Base + Adapter)

In [3]:
# 4-bit Quantization Config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=False,
)

# Load Base Model
base_model = AutoModelForCausalLM.from_pretrained(
    CONF['base_model'],
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)

# Load Adapter
model = PeftModel.from_pretrained(base_model, CONF['adapter_path'])
model.eval()

# Load Tokenizer
tokenizer = AutoTokenizer.from_pretrained(CONF['base_model'], trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

## 3. 데이터 로드 및 평가 함수 정의

In [ ]:
# Load Dev Data for Validation
dev_df = pd.read_csv(os.path.join(CONF['data_path'], 'dev.csv'))
print(f"Dev set size: {len(dev_df)}")

# ROUGE Metric (use `evaluate` instead of deprecated `datasets.load_metric`)
from evaluate import load as load_metric
rouge = load_metric("rouge")


def compute_rouge(predictions, references):
    """ROUGE 점수 계산"""
    results = rouge.compute(predictions=predictions, references=references, use_stemmer=True)
    return {
        "rouge1": results["rouge1"].mid.fmeasure if hasattr(results["rouge1"], "mid") else results["rouge1"],
        "rouge2": results["rouge2"].mid.fmeasure if hasattr(results["rouge2"], "mid") else results["rouge2"],
        "rougeL": results["rougeL"].mid.fmeasure if hasattr(results["rouge1"], "mid") else results["rougeL"],
    }

# ============================================
# 정답 요약 길이 분석
# ============================================
import re

ref_lengths = [len(s.split()) for s in dev_df['summary']]
print(f"\n📊 Reference Summary Length Statistics:")
print(f"  - Mean: {sum(ref_lengths)/len(ref_lengths):.1f} words")
print(f"  - Min: {min(ref_lengths)} words")
print(f"  - Max: {max(ref_lengths)} words")
print(f"  - Median: {sorted(ref_lengths)[len(ref_lengths)//2]} words")

# 길이 분포 시각화를 위한 히스토그램 데이터
length_bins = {}
for l in ref_lengths:
    bin_key = (l // 10) * 10  # 10단위로 그룹화
    length_bins[bin_key] = length_bins.get(bin_key, 0) + 1

print(f"\n📈 Length Distribution (words):")
for bin_start in sorted(length_bins.keys()):
    count = length_bins[bin_start]
    bar = "█" * (count // 2)
    print(f"  {bin_start:3d}-{bin_start+9:3d}: {bar} ({count})")

Dev set size: 499

📊 Reference Summary Length Statistics:
  - Mean: 15.3 words
  - Min: 5 words
  - Max: 54 words
  - Median: 14 words

📈 Length Distribution (words):
    0-  9: █████████████████████████████████████████ (83)
   10- 19: █████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████ (314)
   20- 29: ████████████████████████████████████████ (81)
   30- 39: █████████ (18)
   40- 49: █ (2)
   50- 59:  (1)


## 4. 실험 1: Prompt Engineering
다양한 프롬프트 스타일을 테스트하여 모델이 요약을 더 잘 생성하도록 유도합니다.

In [5]:
# ============================================
# 개선된 프롬프트 (한국어 + 특수토큰 명시)
# ============================================
prompts = {
    "basic": "### User:\nSummarize the following dialogue:\n\n{dialogue}\n\n### Assistant:\n",
    
    "korean": "### User:\n다음 대화를 한국어로 요약해줘:\n\n{dialogue}\n\n### Assistant:\n",
    
    "ko_token": (
        "### User:\n"
        "다음 일상 대화를 한국어로 2~3문장 정도의 자연스러운 평서문으로 요약해줘.\n"
        "- 대화의 핵심 내용과 중요한 사건, 인물의 행동만 포함해.\n"
        "- #Person1#, #Person2#, #PhoneNumber# 같은 특수 토큰은 요약에서도 그대로 사용해.\n\n"
        "{dialogue}\n\n### Assistant:\n"
    ),
}

def run_inference(model, tokenizer, dialogue, prompt_template, params):
    prompt = prompt_template.format(dialogue=dialogue)
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            **params,
            pad_token_id=tokenizer.eos_token_id,
            eos_token_id=tokenizer.eos_token_id
        )
    
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    try:
        summary = generated_text.split("### Assistant:\n")[1].strip()
    except:
        summary = generated_text
    return summary

# ============================================
# 개선된 평가 설정
# ============================================
# (1) Dev 샘플 수: 10개 → 100개 (랜덤 샘플링)
EVAL_SAMPLE_SIZE = 100
eval_df = dev_df.sample(n=min(EVAL_SAMPLE_SIZE, len(dev_df)), random_state=CONF["seed"]).reset_index(drop=True)
print(f"Evaluation sample size: {len(eval_df)}")

# (2) 대회 방식 점수 계산 함수
def compute_final_score(rouge_scores):
    """대회 기준: (R1 + R2 + RL) / 3 * 100"""
    return (rouge_scores["rouge1"] + rouge_scores["rouge2"] + rouge_scores["rougeL"]) / 3 * 100

# ============================================
# 프롬프트 비교 실험 (100개 샘플)
# ============================================
print("=== Prompt Engineering Test (100 samples) ===")
default_params = {"max_new_tokens": 96, "do_sample": False, "repetition_penalty": 1.1}

prompt_results = []
for name, template in prompts.items():
    preds = []
    for dialogue in tqdm(eval_df['dialogue'], desc=f"Prompt [{name}]"):
        preds.append(run_inference(model, tokenizer, dialogue, template, default_params))
    
    scores = compute_rouge(preds, eval_df['summary'].tolist())
    final = compute_final_score(scores)
    prompt_results.append({"prompt": name, **scores, "final_score": final})
    print(f"Prompt [{name}]: R1={scores['rouge1']:.4f}, R2={scores['rouge2']:.4f}, RL={scores['rougeL']:.4f}, Final={final:.2f}")

# 최적 프롬프트 선택
best_prompt_name = max(prompt_results, key=lambda x: x["final_score"])["prompt"]
best_prompt = prompts[best_prompt_name]
print(f"\n✅ Best Prompt: {best_prompt_name}")

Evaluation sample size: 100
=== Prompt Engineering Test (100 samples) ===


Prompt [basic]: 100%|██████████| 100/100 [29:28<00:00, 17.68s/it]


Prompt [basic]: R1=0.7660, R2=0.4936, RL=0.7226, Final=66.07


Prompt [korean]: 100%|██████████| 100/100 [32:02<00:00, 19.23s/it]


Prompt [korean]: R1=0.7777, R2=0.5279, RL=0.7405, Final=68.21


Prompt [ko_token]: 100%|██████████| 100/100 [40:17<00:00, 24.17s/it]

Prompt [ko_token]: R1=0.7915, R2=0.5178, RL=0.7514, Final=68.69

✅ Best Prompt: ko_token


## 5. 실험 2: Decoding Strategies (Grid Search)
Beam Search, Repetition Penalty, No Repeat N-gram Size 등을 조합하여 최적의 파라미터를 찾습니다.

In [6]:
# ============================================
# 디코딩 파라미터 그리드 서치 (핵심 4개 조합만)
# ============================================
# 제안된 최소 실험 세트:
# - repetition_penalty: 1.0 vs 1.1
# - num_beams: 1 vs 3

search_space = [
    {"name": "A: beam=1, rep=1.1", "params": {"max_new_tokens": 96, "num_beams": 1, "do_sample": False, "repetition_penalty": 1.1}},
    {"name": "B: beam=3, rep=1.1", "params": {"max_new_tokens": 96, "num_beams": 3, "early_stopping": True, "repetition_penalty": 1.1}},
    {"name": "C: beam=1, rep=1.0", "params": {"max_new_tokens": 96, "num_beams": 1, "do_sample": False, "repetition_penalty": 1.0}},
    {"name": "D: beam=3, rep=1.0", "params": {"max_new_tokens": 96, "num_beams": 3, "early_stopping": True, "repetition_penalty": 1.0}},
]

print(f"=== Decoding Strategy Search (using prompt: {best_prompt_name}) ===")
print(f"Evaluating on {len(eval_df)} samples\n")

decoding_results = []
for config in search_space:
    preds = []
    lengths = []
    
    for dialogue in tqdm(eval_df['dialogue'], desc=config['name']):
        summary = run_inference(model, tokenizer, dialogue, best_prompt, config['params'])
        preds.append(summary)
        lengths.append(len(summary.split()))
    
    scores = compute_rouge(preds, eval_df['summary'].tolist())
    final = compute_final_score(scores)
    avg_len = sum(lengths) / len(lengths)
    
    decoding_results.append({
        "config": config['name'], 
        **scores, 
        "final_score": final,
        "avg_length": avg_len
    })
    print(f"Config [{config['name']}]: R1={scores['rouge1']:.4f}, R2={scores['rouge2']:.4f}, RL={scores['rougeL']:.4f}, Final={final:.2f}, AvgLen={avg_len:.1f}")

# 최적 디코딩 설정 선택
best_decoding = max(decoding_results, key=lambda x: x["final_score"])
print(f"\n✅ Best Decoding Config: {best_decoding['config']}")
print(f"   Final Score: {best_decoding['final_score']:.2f}")
print(f"   Avg Length: {best_decoding['avg_length']:.1f} words")

=== Decoding Strategy Search (using prompt: ko_token) ===
Evaluating on 100 samples



A: beam=1, rep=1.1: 100%|██████████| 100/100 [40:15<00:00, 24.15s/it]


Config [A: beam=1, rep=1.1]: R1=0.7915, R2=0.5178, RL=0.7514, Final=68.69, AvgLen=14.8


B: beam=3, rep=1.1: 100%|██████████| 100/100 [1:56:49<00:00, 70.09s/it]  


Config [B: beam=3, rep=1.1]: R1=0.7416, R2=0.4823, RL=0.6978, Final=64.06, AvgLen=15.9


C: beam=1, rep=1.0: 100%|██████████| 100/100 [43:25<00:00, 26.06s/it]


Config [C: beam=1, rep=1.0]: R1=0.7715, R2=0.5185, RL=0.7374, Final=67.58, AvgLen=16.2


D: beam=3, rep=1.0: 100%|██████████| 100/100 [1:57:03<00:00, 70.24s/it]  

Config [D: beam=3, rep=1.0]: R1=0.7445, R2=0.4891, RL=0.6993, Final=64.43, AvgLen=16.2

✅ Best Decoding Config: A: beam=1, rep=1.1
   Final Score: 68.69
   Avg Length: 14.8 words


## 5-1. 최적 파라미터 찾기: max_new_tokens 그리드 서치
현재 문제를 해결하기 위해 max_new_tokens을 128, 150, 160으로 테스트하고 추가 파라미터도 조정합니다.


In [11]:
# ============================================
# 최적 파라미터 찾기: max_new_tokens 그리드 서치
# ============================================
# 문제: 현재 max_new_tokens=96은 너무 짧음 (평균 14.9 단어 생성)
# 정답 요약 평균: ~30-50 단어 필요
# 
# 테스트할 설정:
# 1. max_new_tokens: 128, 150, 160
# 2. num_beams: 1, 3
# 3. length_penalty: 1.0, 1.2 (더 긴 요약 선호)

print(f"=== 최적 파라미터 찾기 (Dev 20개 샘플) ===")
print(f"현재 reference 평균 길이: {sum(ref_lengths)/len(ref_lengths):.1f} 단어\n")

# 작은 샘플로 빠르게 테스트 (20개만)
test_size = 20
test_df_small = eval_df.sample(n=test_size, random_state=CONF['seed']).reset_index(drop=True)

search_configs = [
    # 기존 설정 (비교용)
    {
        "name": "현재 (max=96, beam=1, rep=1.1)",
        "params": {"max_new_tokens": 96, "num_beams": 1, "do_sample": False, "repetition_penalty": 1.1}
    },
    # 새로운 설정들
    {
        "name": "개선1 (max=128, beam=1, rep=1.1)",
        "params": {"max_new_tokens": 128, "num_beams": 1, "do_sample": False, "repetition_penalty": 1.1}
    },
    {
        "name": "개선2 (max=150, beam=1, rep=1.1)",
        "params": {"max_new_tokens": 150, "num_beams": 1, "do_sample": False, "repetition_penalty": 1.1}
    },
    {
        "name": "개선3 (max=160, beam=1, rep=1.0, len=1.0)",
        "params": {"max_new_tokens": 160, "num_beams": 1, "do_sample": False, "repetition_penalty": 1.0, "length_penalty": 1.0}
    },
    {
        "name": "개선4 (max=150, beam=3, rep=1.1, len=1.2)",
        "params": {"max_new_tokens": 150, "num_beams": 3, "early_stopping": True, "repetition_penalty": 1.1, "length_penalty": 1.2}
    },
    {
        "name": "개선5 (max=160, beam=3, rep=1.0, len=1.2)",
        "params": {"max_new_tokens": 160, "num_beams": 3, "early_stopping": True, "repetition_penalty": 1.0, "length_penalty": 1.2}
    },
]

param_search_results = []

for config in search_configs:
    preds = []
    lengths = []
    
    for dialogue in tqdm(test_df_small['dialogue'], desc=config['name']):
        summary = run_inference(model, tokenizer, dialogue, best_prompt, config['params'])
        summary = post_process(summary)
        preds.append(summary)
        lengths.append(len(summary.split()))
    
    scores = compute_rouge(preds, test_df_small['summary'].tolist())
    final = compute_final_score(scores)
    avg_len = sum(lengths) / len(lengths)
    
    param_search_results.append({
        "config": config['name'],
        "rouge1": scores['rouge1'],
        "rouge2": scores['rouge2'],
        "rougeL": scores['rougeL'],
        "final_score": final,
        "avg_length": avg_len
    })
    
    print(f"\n[{config['name']}]")
    print(f"  R1: {scores['rouge1']:.4f} | R2: {scores['rouge2']:.4f} | RL: {scores['rougeL']:.4f}")
    print(f"  Final: {final:.2f} | Avg Length: {avg_len:.1f} 단어")

# 최적 설정 선택 (final_score 기준)
best_config_idx = max(range(len(param_search_results)), key=lambda i: param_search_results[i]['final_score'])
best_config = param_search_results[best_config_idx]

print("\n" + "="*80)
print(f"✅ 최적 설정 선택: {search_configs[best_config_idx]['name']}")
print(f"   예상 Final Score: {best_config['final_score']:.2f}")
print(f"   평균 길이: {best_config['avg_length']:.1f} 단어")
print("="*80)


=== 최적 파라미터 찾기 (Dev 20개 샘플) ===
현재 reference 평균 길이: 15.3 단어



현재 (max=96, beam=1, rep=1.1): 100%|██████████| 20/20 [08:40<00:00, 26.01s/it]


=== 최적 파라미터 찾기 (Dev 20개 샘플) ===
현재 reference 평균 길이: 15.3 단어



현재 (max=96, beam=1, rep=1.1): 100%|██████████| 20/20 [08:40<00:00, 26.01s/it]



[현재 (max=96, beam=1, rep=1.1)]
  R1: 0.8186 | R2: 0.4533 | RL: 0.7711
  Final: 68.10 | Avg Length: 16.1 단어


...


[개선3 (max=160, beam=1, rep=1.0, len=1.0)]
  R1: 0.7935 | R2: 0.4847 | RL: 0.7614
  Final: 67.99 | Avg Length: 19.6 단어


개선4 (max=150, beam=3, rep=1.1, len=1.2): 100%|██████████| 20/20 [31:50<00:00, 95.53s/it] 




[개선4 (max=150, beam=3, rep=1.1, len=1.2)]
  R1: 0.7049 | R2: 0.4387 | RL: 0.6929
  Final: 61.22 | Avg Length: 21.6 단어


개선5 (max=160, beam=3, rep=1.0, len=1.2): 100%|██████████| 20/20 [33:24<00:00, 100.24s/it]


[개선5 (max=160, beam=3, rep=1.0, len=1.2)]
  R1: 0.7119 | R2: 0.3805 | RL: 0.6997
  Final: 59.73 | Avg Length: 21.6 단어

✅ 최적 설정 선택: 현재 (max=96, beam=1, rep=1.1)
   예상 Final Score: 68.10
   평균 길이: 16.1 단어


## 6. 51.6863 초과를 위한 파라미터 탐색
기준점(51.6863)을 넘기 위해 다양한 파라미터 조합을 Dev 샘플에서 테스트합니다.

In [12]:
# ============================================
# 🎯 51.6863 초과를 위한 파라미터 탐색
# ============================================
# 기준점: 51.6863 (basic, 150, beam=3, rep=1.2, len=1.2)
# 
# 실험 전략:
#   - length_penalty: 1.1, 1.3
#   - num_beams: 4
#   - repetition_penalty: 1.15

import re

def post_process(summary: str) -> str:
    """추론 후 요약 결과 정리"""
    summary = re.sub(r"\s+", " ", summary).strip()
    summary = summary.replace("### User:", "").replace("### Assistant:", "").strip()
    if not summary:
        return "요약을 생성할 수 없습니다."
    return summary

print("=" * 60)
print("🔬 51.6863 초과를 위한 파라미터 탐색")
print("=" * 60)

# Dev 샘플로 빠른 테스트 (50개)
QUICK_EVAL_SIZE = 50
quick_eval_df = dev_df.sample(n=QUICK_EVAL_SIZE, random_state=42)

def quick_evaluate(params, prompt_name="basic"):
    """빠른 Dev 샘플 평가"""
    prompt = prompts[prompt_name]
    preds = []
    for dialogue in tqdm(quick_eval_df['dialogue'], desc=f"Testing"):
        raw = run_inference(model, tokenizer, dialogue, prompt, params)
        clean = post_process(raw)
        preds.append(clean)
    
    refs = quick_eval_df['summary'].tolist()
    scores = compute_rouge(preds, refs)
    final = (scores["rouge1"] + scores["rouge2"] + scores["rougeL"]) / 3 * 100
    return final, scores, preds

# 기준 설정 (51.6863 설정)
baseline_params = {
    "max_new_tokens": 150,
    "num_beams": 3,
    "early_stopping": True,
    "no_repeat_ngram_size": 0,
    "repetition_penalty": 1.2,
    "length_penalty": 1.2,
}

# 실험 설정들
experiments = [
    ("기준점 (len=1.2, beam=3, rep=1.2)", {**baseline_params}),
    ("실험1: len=1.1", {**baseline_params, "length_penalty": 1.1}),
    ("실험2: len=1.3", {**baseline_params, "length_penalty": 1.3}),
    ("실험3: beam=4", {**baseline_params, "num_beams": 4}),
    ("실험4: rep=1.15", {**baseline_params, "repetition_penalty": 1.15}),
    ("실험5: len=1.1 + beam=4", {**baseline_params, "length_penalty": 1.1, "num_beams": 4}),
]

results = []
for name, params in experiments:
    print(f"\n▶ {name}")
    final, scores, preds = quick_evaluate(params)
    avg_len = sum(len(p.split()) for p in preds) / len(preds)
    results.append({
        "name": name,
        "params": params,
        "final_score": final,
        "rouge1": scores["rouge1"] * 100,
        "rouge2": scores["rouge2"] * 100,
        "rougeL": scores["rougeL"] * 100,
        "avg_len": avg_len
    })
    print(f"   Final: {final:.2f} | R1: {scores['rouge1']*100:.2f} | R2: {scores['rouge2']*100:.2f} | RL: {scores['rougeL']*100:.2f} | Len: {avg_len:.1f}")

# 결과 정렬 (점수 높은 순)
print("\n" + "=" * 60)
print("📊 결과 요약 (점수 높은 순)")
print("=" * 60)
baseline_score = results[0]["final_score"]
for r in sorted(results, key=lambda x: x["final_score"], reverse=True):
    diff = r["final_score"] - baseline_score
    marker = "🏆" if r["final_score"] == max(x["final_score"] for x in results) else "  "
    print(f"{marker} {r['name']}: {r['final_score']:.2f} ({diff:+.2f}) | Len: {r['avg_len']:.1f}")

# 최고 설정 출력
best_result = max(results, key=lambda x: x["final_score"])
print(f"\n✅ 최고 설정: {best_result['name']}")
print(f"   Dev Score: {best_result['final_score']:.2f}")
print(f"   Params: {best_result['params']}")

🔬 51.6863 초과를 위한 파라미터 탐색

▶ 기준점 (len=1.2, beam=3, rep=1.2)


Testing: 100%|██████████| 50/50 [49:33<00:00, 59.48s/it] 



   Final: 75.27 | R1: 83.41 | R2: 62.34 | RL: 80.06 | Len: 15.9

▶ 실험1: len=1.1


Testing: 100%|██████████| 50/50 [49:35<00:00, 59.52s/it] 



   Final: 75.27 | R1: 83.41 | R2: 62.34 | RL: 80.06 | Len: 15.8

▶ 실험2: len=1.3


...

   Final: 73.81 | R1: 82.58 | R2: 59.94 | RL: 78.91 | Len: 15.8

▶ 실험4: rep=1.15


Testing: 100%|██████████| 50/50 [50:46<00:00, 60.93s/it] 



   Final: 73.25 | R1: 81.77 | R2: 59.64 | RL: 78.35 | Len: 16.6

▶ 실험5: len=1.1 + beam=4


Testing:   4%|▍         | 2/50 [02:16<54:25, 68.03s/it]



KeyboardInterrupt: 

## 7. 최종 추론 및 제출 파일 생성
위 실험에서 얻은 최적의 파라미터와 프롬프트를 적용하여 Test 셋에 대한 추론을 수행합니다.

In [ ]:
# ============================================
# 최종 추론 설정 (51.6863점 최고 설정 기반)
# ============================================
best_prompt_name = "basic"
FINAL_PROMPT = prompts["basic"]

# 최적화된 디코딩 파라미터
FINAL_PARAMS = {
    "max_new_tokens": 150,           # 한국어 요약에 적절한 길이
    "num_beams": 3,                  # beam search 활성화
    "early_stopping": True,          # beam search 최적화
    "no_repeat_ngram_size": 0,       # 중복 제거 비활성화
    "repetition_penalty": 1.2,       # 반복 억제
    "length_penalty": 1.3,           # 요약 길이 조절
}

# ============================================
# 후처리 함수 (추론 결과 정리용)
# ============================================
def post_process(summary: str) -> str:
    """
    추론 후 요약 결과 정리:
    1. 공백 정리 ✅
    2. 프롬프트 에코 제거 ✅
    3. 문장 분리 & 3문장 제한 ❌ 제거 (ROUGE 손실 방지)
    """
    # 1) 공백 정리
    summary = re.sub(r"\s+", " ", summary).strip()
    
    # 2) 프롬프트 에코 제거
    summary = summary.replace("### User:", "").replace("### Assistant:", "").strip()
    
    # 3) 빈 결과 방지
    if not summary:
        return "요약을 생성할 수 없습니다."
    
    return summary

# ============================================
# Test 셋 추론
# ============================================
test_df = pd.read_csv(os.path.join(CONF['data_path'], 'test.csv'))

print(f"Starting Final Inference with:")
print(f"  - Prompt: {best_prompt_name}")
print(f"  - Params: {FINAL_PARAMS}")
print(f"  - Test set size: {len(test_df)}")

final_summaries = []
final_lengths = []

for dialogue in tqdm(test_df['dialogue'], desc="Final Inference"):
    raw_summary = run_inference(model, tokenizer, dialogue, FINAL_PROMPT, FINAL_PARAMS)
    clean_summary = post_process(raw_summary)  # 추론 후 후처리
    final_summaries.append(clean_summary)
    final_lengths.append(len(clean_summary.split()))

# 결과 저장
submission = pd.DataFrame({
    'fname': test_df['fname'],
    'summary': final_summaries
})

output_path = "./prediction/submit_solar_v3.csv"
submission.to_csv(output_path, index=False)

# ============================================
# 통계 출력
# ============================================
print(f"\n✅ Saved to {output_path}")
print(f"\n📊 Generated Summary Statistics:")
print(f"  - Total: {len(final_summaries)}")
print(f"  - Avg Length: {sum(final_lengths)/len(final_lengths):.1f} words")
print(f"  - Min Length: {min(final_lengths)} words")
print(f"  - Max Length: {max(final_lengths)} words")

# 정답 길이와 비교
ref_avg = sum(ref_lengths) / len(ref_lengths)
gen_avg = sum(final_lengths) / len(final_lengths)
print(f"\n📏 Length Comparison:")
print(f"  - Reference Avg: {ref_avg:.1f} words")
print(f"  - Generated Avg: {gen_avg:.1f} words")
print(f"  - Difference: {gen_avg - ref_avg:+.1f} words ({(gen_avg/ref_avg - 1)*100:+.1f}%)")

Starting Final Inference with:
  - Prompt: basic
  - Params: {'max_new_tokens': 150, 'num_beams': 3, 'early_stopping': True, 'no_repeat_ngram_size': 0, 'repetition_penalty': 1.2, 'length_penalty': 1.3}
  - Test set size: 499


Final Inference: 100%|██████████| 499/499 [8:50:47<00:00, 63.82s/it]    


✅ Saved to ./prediction/submit_solar_v3.csv

📊 Generated Summary Statistics:
  - Total: 499
  - Avg Length: 16.1 words
  - Min Length: 6 words
  - Max Length: 35 words

📏 Length Comparison:
  - Reference Avg: 15.3 words
  - Generated Avg: 16.1 words
  - Difference: +0.8 words (+5.0%)


: 